## Out.txt

In [6]:
import json
import pandas as pd
import numpy as np


# CHANNEL_LIST_FILE = "data/channel_list_merged/finance_kw_10k_sub_+.csv"
# CHANNEL_QUALITY_FILE = "data/channel_quality/finance_kw_10k_sub_+_filter_0-200.jsonl"

CHANNEL_LIST_FILE = "data/channel_list_merged/starngage_5000.csv"
CHANNEL_QUALITY_FILE = "data/channel_quality/starngage_5000_filter_0-100.jsonl"

In [4]:
df = pd.read_csv(CHANNEL_LIST_FILE)

# # remove duplicates
df = df.drop_duplicates()
len(df[df['n_subs'] > 10000])

4528

In [7]:
rs = []
with open(CHANNEL_QUALITY_FILE, "r") as f:
    r = f.readlines()
    for i in r:
        # rs.append(json.loads(i.replace("'", "\"").replace("False", "false").replace("True", "true")))
        rs.append(json.loads(i))

In [8]:

def clean_channel_cond(r):
    n_total = len(r['snrss'])
    n_snr_ok = (np.array(r['snrss']) > 20).sum()
    # n_ac_ok = (np.array(r['acss'])).sum()
    ac_speech_probs = []
    for ac_seg in r['acss']:
        for item in ac_seg:
            if item['label'] == 'Speech':
                ac_speech_probs.append(item['score'])
    n_ac_ok = (np.array(ac_speech_probs) > 0.9).sum()

    mean_snr = np.mean(r['snrss'])
    mean_ac = np.mean(ac_speech_probs)

    is_good = n_total > 0 and n_snr_ok / n_total > 0.1 and n_ac_ok / n_total > 0.1
    return is_good, {
        'n_total': n_total,
        'n_snr_ok': n_snr_ok,
        'n_ac_ok': n_ac_ok,
        'mean_snr': mean_snr,
        'mean_ac': mean_ac,
        'url': r['url'],
    }

r_ok = []
n_ok = []
r_not_ok = []
n_not_ok = []
for r in rs:
    is_good, info = clean_channel_cond(r)
    if is_good:
        r_ok.append(r)
        n_ok.append(info)
    else:
        r_not_ok.append(r)
        n_not_ok.append(info)

d:\Projects\SE-datapipeline\venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Projects\SE-datapipeline\venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
ok_df = df[df['url'].isin([r['url'] for r in r_ok])]
not_ok_df = df[df['url'].isin([r['url'] for r in r_not_ok])]

# drop duplicate title
ok_df = ok_df.drop_duplicates(subset=['title'])
not_ok_df = not_ok_df.drop_duplicates(subset=['title'])

pass

In [10]:
len(ok_df)

1

In [11]:
len(not_ok_df)

99

In [12]:
# join n_total, n_snr_ok, n_ac_ok
ok_df = pd.merge(ok_df, pd.DataFrame(n_ok), left_on='url', right_on='url')
not_ok_df = pd.merge(not_ok_df, pd.DataFrame(n_not_ok), left_on='url', right_on='url')

# move column 'url' to the last
cols = list(ok_df.columns)
cols.remove('url')
cols.append('url')
ok_df = ok_df[cols]
not_ok_df = not_ok_df[cols]

In [14]:
ok_df.sort_values(by='mean_snr', ascending=False)

,title,id,n_videos,n_views,n_subs,custom_url,email,n_total,n_snr_ok,n_ac_ok,mean_snr,mean_ac,url
0,Truyen Hinh Vinh Long,UCYIUuKyiDCBnG-tp6jr-1nA,28972,3843925280,5450000,@vinhlongtv,banbientap@thvl.vn,186,62,21,16.883611,0.772559,https://www.youtube.com/channel/UCYIUuKyiDCBnG...


In [19]:
# not_ok_df order by mean snr 
# not_ok_df.sort_values(by='mean_ac', ascending=False).sort_values(by='mean_snr', ascending=False).head(20)
not_ok_df.sort_values(by='n_ac_ok', ascending=False).head(50)


,title,id,n_videos,n_views,n_subs,custom_url,email,n_total,n_snr_ok,n_ac_ok,mean_snr,mean_ac,url
94,Vie GIẢITRÍ,UC2Lgi2uPsOcCVF3imz7l2mg,7486,1181579293,2830000,viegiaitri,khoa.nguyen@viechannel.vn,261,24,41,12.244121,0.692973,https://www.youtube.com/channel/UC2Lgi2uPsOcCV...
60,TOÀN CẢNH 24H,UCZcCmWhvK0gfThyWRmRerDQ,76008,5235420725,5350000,@toancanh24,NaN,303,157,19,22.281781,0.800317,https://www.youtube.com/channel/UCZcCmWhvK0gfT...
61,BNC Now,UC7723FqVehXq2zeRb3tP0hQ,76065,2768927696,3820000,@bncnow,fbncvn@gmail.com,196,20,14,15.431333,0.757004,https://www.youtube.com/channel/UC7723FqVehXq2...
73,PEWPEW,UCsaMa3VD1I9G952DDlOX7aw,2624,354448651,3600000,@pewpew,pewpew.studio.vn@gmail.com,130,50,12,19.356378,0.778160,https://www.youtube.com/channel/UCsaMa3VD1I9G9...
11,Gãy Media,UCTp_WPPxWCjdlXK9kqzxm0A,1128,5026266472,5570000,gaymedia,gaymedia88@gmail.com,288,108,9,19.188504,0.699434,https://www.youtube.com/channel/UCTp_WPPxWCjdl...
45,POPS Kids,UC5ezaYrzZpyItPSRG27MLpg,3673,12385888612,18100000,@popskids,NaN,205,43,9,17.058227,0.756870,https://www.youtube.com/channel/UC5ezaYrzZpyIt...
41,PHD Sports,UC7mc9-vty5WAeIYi0PIkONg,1104,4915880764,8060000,@phdsportsvn,phdgroupmedia@gmail.com,333,10,7,10.796091,0.702860,https://www.youtube.com/channel/UC7mc9-vty5WAe...
87,Phong Cận Tv,UCwHsswGDpVWHPAC-ssScIwQ,1093,2223263885,4270000,@phongcantv102,NaN,132,16,6,13.827519,0.649692,https://www.youtube.com/channel/UCwHsswGDpVWHP...
32,Khoa Pug,UCAGikSCgqbeIXCvoXixWovQ,543,1061214830,4660000,@khoapug_tv,khoapug.booking@gmail.com,312,39,5,14.309625,0.741000,https://www.youtube.com/channel/UCAGikSCgqbeIX...
59,THẾ GIỚI NGHIÊNG,UCHPzpxcYhxkb0fMJKeSe66g,4192,308452767,4010000,@thegioinghieng,NaN,288,6,5,13.063932,0.799150,https://www.youtube.com/channel/UCHPzpxcYhxkb0...


In [ ]:
all_df = pd.concat([ok_df, not_ok_df])
all_df['score'] = all_df['n_ac_ok'] / all_df['n_total']

In [10]:
ok_df.to_csv('tmp_16_channel_ok.csv', index=False)

In [11]:
# move column 'url' to the last
cols = list(ok_df.columns)
cols.remove('url')
cols.append('url')
ok_df = ok_df[cols]
not_ok_df = not_ok_df[cols]

ok_df.to_csv('channel_clean.csv', index=False)
not_ok_df.to_csv('channel_noise.csv', index=False)